In [5]:
import xgboost
import shap
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve, roc_curve
import numpy as np
import matplotlib.pyplot as plt
import plots as plots
import pickle
import lime
import lime.lime_tabular

/opt/homebrew/Caskroom/miniforge/base/envs/xai/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df = pd.read_csv('raw-dataset.csv')

In [3]:
df.head()

,Unnamed: 0,Test ID,easting__N,northing__,GWD (m),Elevation,L (m),Slope (%),Displ (m),PGA (g),Qtncs_med,Qtncs_std,Ic_med,Ic_std
0,0,15635.0,1.577647e+06,5.182056e+06,1.300896,1.123009,211.770466,0.905948,0.195432,0.532398,123.393393,23.853785,1.625464,0.112291
1,1,15629.0,1.577685e+06,5.181809e+06,1.788212,2.044325,115.795053,0.451034,0.239763,0.542307,135.790982,34.965532,1.626258,0.358778
2,2,183.0,1.577701e+06,5.181709e+06,1.637517,2.003797,137.265348,0.941866,0.377358,0.545784,129.140240,38.641063,4.569982,0.434698
3,3,158.0,1.577726e+06,5.181965e+06,1.165643,0.992154,73.885082,2.699290,0.241755,0.535794,94.733152,32.039927,1.766342,0.462708
4,4,180.0,1.577731e+06,5.182267e+06,2.121364,1.933611,119.745095,1.187781,0.156150,0.528980,94.189786,9.524548,1.864985,0.162700


In [6]:
data = pickle.load(open('xgb_models/data5.pkl', 'rb'))
model = pickle.load(open('xgb_models/opt_XGB_YN0.pkl', 'rb'))

X_train_target = data['X_train_target']
X_train_target = X_train_target.drop(['PGA (g)', 'Elevation', '4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)
X_test_target = data['X_test_target']
X_test_target = X_test_target.drop(['PGA (g)', 'Elevation', '4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)
X = data['X']
X = X.drop(['PGA (g)', 'Elevation', '4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)

X_train = data['X_train']
X_train = X_train.drop(['PGA (g)', 'Elevation', '4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)
X_test = data['X_test']
X_test = X_test.drop(['PGA (g)', 'Elevation', '4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)
X_val = data['X_val']
X_val = X_val.drop(['PGA (g)', 'Elevation', '4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)
X_all = data['X_all']
X_all = X_all.drop(['PGA (g)', 'Elevation', '4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)

y_train = data['y_train']
y_test = data['y_test']
y_val = data['y_val']
y = data['y']

In [8]:
X_train_target.head()

,Test ID,GWD (m),L (km),Slope (%),Target
6043,17633.0,1.718826,0.535128,0.397376,0
5613,43138.0,1.518877,0.380435,0.459704,1
4002,11169.0,1.316730,0.234274,0.623473,0
1067,54829.0,2.401471,1.718972,2.861161,0
854,22518.0,2.585827,0.585417,0.030798,0


In [32]:
len(X_train_target.loc[X_train_target['L (km)'] > 2].loc[X_train_target['Target']==1])

55

In [34]:
X_train_target.loc[X_train_target['GWD (m)'] > 3]

,Test ID,GWD (m),L (km),Slope (%),Target
1249,2903.0,3.822122,1.347567,1.094091,0
1226,5545.0,3.627551,1.357141,1.161011,0
1582,54603.0,3.369406,1.401521,2.117453,0
322,33319.0,3.130784,0.808296,0.957238,0
276,49091.0,3.041477,0.618659,1.940264,1
...,...,...,...,...,...
1505,54561.0,3.073484,0.795188,2.218553,0
560,22265.0,3.696963,1.746468,1.889506,0
1206,4317.0,4.459133,1.311164,1.328388,0
430,25706.0,3.268571,1.283932,1.445562,0


In [29]:
L_03 = X_train_target.loc[X_train_target['L (km)'] < 0.3] 
L_03.loc[L_03['Slope (%)'] > 2].loc[L_03['Target']==1]

,Test ID,GWD (m),L (km),Slope (%),Target
1465,49827.0,2.852315,0.221454,4.466954,1
5296,120.0,0.757456,0.007051,2.633182,1
5288,119.0,0.812953,0.011328,3.519404,1
4024,38304.0,1.593081,0.044541,2.150381,1
5703,1133.0,2.151764,0.011646,4.964096,1
...,...,...,...,...,...
5298,525.0,1.110009,0.009192,4.109590,1
5311,15615.0,2.866079,0.202173,2.893787,1
6190,56.0,1.783620,0.018390,3.176269,1
5726,49.0,1.390997,0.017915,2.341863,1


In [21]:
gwd_2 = X_train_target.loc[X_train_target['GWD (m)'] > 1.95].loc[X_train_target['GWD (m)'] < 2.05]
gwd_2.head()

,Test ID,GWD (m),L (km),Slope (%),Target
159,47844.0,1.995111,0.697448,0.997108,0
6071,25608.0,2.020858,0.652842,1.024892,0
6023,23982.0,2.026702,0.471795,0.997204,0
3548,2911.0,1.999451,1.518939,0.228218,1
4996,42300.0,2.007189,1.281995,0.674000,0


In [24]:
gwd_2.loc[gwd_2['L (km)']<0.7]

,Test ID,GWD (m),L (km),Slope (%),Target
159,47844.0,1.995111,0.697448,0.997108,0
6071,25608.0,2.020858,0.652842,1.024892,0
6023,23982.0,2.026702,0.471795,0.997204,0
6412,42823.0,2.038535,0.281955,0.452543,1
6345,45.0,1.958730,0.138996,0.199778,1
...,...,...,...,...,...
3724,18581.0,2.008337,0.385705,2.016310,1
6371,27352.0,2.038247,0.156816,0.215935,1
201,49593.0,1.960299,0.331011,0.346281,1
2927,22391.0,1.994074,0.552113,0.158368,1


# Model B

In [35]:
data = pickle.load(open('xgb_models/data5.pkl', 'rb'))
model = pickle.load(open('xgb_models/opt_XGB_YN3.pkl', 'rb'))

X_train_target = data['X_train_target']
X_train_target = X_train_target.drop(['4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)
X_test_target = data['X_test_target']
X_test_target = X_test_target.drop(['4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)
X = data['X']
X = X.drop(['4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)

X_train = data['X_train']
X_train = X_train.drop(['4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)
X_test = data['X_test']
X_test = X_test.drop(['4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)
X_val = data['X_val']
X_val = X_val.drop(['4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)
X_all = data['X_all']
X_all = X_all.drop(['4m_Qtncs_m', '4m_Qtncs_s', '4m_Ic_med', '4m_Ic_std'], axis=1)

y_train = data['y_train']
y_test = data['y_test']
y_val = data['y_val']
y = data['y']

In [73]:
X_train_target.loc[X_train_target['PGA (g)'] > 0.5 ].loc[X_train_target['Elevation'] < 3]

,Test ID,GWD (m),Elevation,L (km),Slope (%),PGA (g),Target
4002,11169.0,1.316730,1.710117,0.234274,0.623473,0.508720,0
6509,26078.0,2.638395,1.992321,0.614216,1.473299,0.505761,1
6492,33300.0,2.814786,2.109118,0.817724,1.418949,0.509021,0
4007,50902.0,1.244507,1.521190,0.136821,0.097995,0.510675,0
4017,21524.0,1.418289,1.136799,0.044932,1.198898,0.515968,0
...,...,...,...,...,...,...,...
3489,27702.0,1.592636,1.128048,0.540976,0.476727,0.508112,1
3957,32694.0,1.467198,1.330797,0.102713,0.319923,0.513281,1
6508,26077.0,2.577652,2.157960,0.536088,0.184244,0.502483,0
1073,37277.0,1.104782,2.864114,1.860314,0.264779,0.522147,0


In [77]:
X_train_target.loc[X_train_target['PGA (g)'] < 0.4 ].loc[X_train_target['Elevation'] > 4].loc[X_train_target['Target'] == 1]

,Test ID,GWD (m),Elevation,L (km),Slope (%),PGA (g),Target
1672,17857.0,2.712797,4.748068,1.209376,2.637378,0.392869,0
4122,42273.0,1.119806,4.503771,2.342682,1.044677,0.377765,1
4269,15252.0,1.522965,4.692791,1.951374,0.804761,0.389010,0
4629,41786.0,1.404957,4.822272,2.048569,0.315132,0.382741,0
6660,16843.0,0.741909,5.025122,1.684591,1.556392,0.390029,1
...,...,...,...,...,...,...,...
4564,25198.0,1.795616,5.614668,2.277463,0.934444,0.365371,0
6654,41496.0,2.316240,6.570876,1.332860,2.063153,0.396269,0
4930,15920.0,2.190864,4.607725,1.312416,0.660428,0.399937,1
1740,14291.0,2.657911,4.230089,1.311970,1.896533,0.395345,0
